In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import sys
sys.path.append('/Users/arthurbabey/Documents/master2/ML/road66/scripts')
from helpers import *
import keras 
import tensorflow as tf


Using TensorFlow backend.


In [2]:
from __future__ import print_function

from keras import regularizers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras import regularizers

In [3]:
# Loaded a set of images
root_dir = "../Datasets/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) # Use all images
print("Loading " + str(n) + " testing images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " groundtruth")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]

Loading 100 testing images
Loading 100 groundtruth


In [4]:
from scipy import ndimage

def get_rotated_images(images, angles):
    rotated_images = [None]*(len(images)*len(angles))
    i = 0
    for angle in angles:
        for image in images:
            rotated_images[i] = ndimage.rotate(image, angle, mode='reflect', order=0, reshape=False)
            i += 1
    return rotated_images

In [5]:
angles = [15,45,60]

rotated_imgs = get_rotated_images(imgs, angles)
gt_rotated_imgs = get_rotated_images(gt_imgs, angles)


# **Concatenate original images and the rotated images**

imgs = imgs + rotated_imgs 
gt_imgs = gt_imgs + gt_rotated_imgs 

In [6]:
# 
# Preparation training data
# 
n = len(imgs)

# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

img_patches = np.asarray(img_patches)
gt_patches = np.asarray(gt_patches)

x_train = img_patches
y_train = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(gt_patches.shape[0])])

print("62500 patches of 16x16 pixels for training are stored in x_train")
print("62500 binary values (1 = road, 0 = bckgrnd) are stored in y_train")

62500 patches of 16x16 pixels for training are stored in x_train
62500 binary values (1 = road, 0 = bckgrnd) are stored in y_train


In [7]:
# 
# Preparation testing data to make a submission
# 

# 
# Ordering image
# 
root_testdir = "../Datasets/test_set_images"
test_names = os.listdir(root_testdir)

num_test = len(test_names)
order = [int(test_names[i].split("_")[1]) for i in range(num_test)]
index = np.argsort(order)

# Load image and reorder them
imgs_test = [load_image(os.path.join(root_testdir, test_names[i], test_names[i]) + ".png") 
             for i in range(num_test)]
imgs_test = [imgs_test[i] for i in index]

# Crop images in patch and Linearize list of patches
patch_size = 16 # each patch is 16*16 pixels

img_patches_test = [img_crop(imgs_test[i], patch_size, patch_size) for i in range(num_test)]


# Linearize list of patches
y_test = np.asarray([img_patches_test[i][j] for i in range(len(img_patches_test)) 
                     for j in range(len(img_patches_test[i]))])

print("72200 patches of 16x16 pixels from the test set are stored in y_test")

72200 patches of 16x16 pixels from the test set are stored in y_test


In [8]:
split_data = True 
seed = 42

#split the x_train and y_train if True

if split_data:

    from sklearn.model_selection import train_test_split

    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state= seed)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)

In [9]:
im_train, im_test, gt_train, gt_test = train_test_split(img_patches, gt_patches, test_size=0.33, random_state= seed)


In [8]:
#arranging y to be used in the CNN 

y_train = np.array(y_train, ndmin = 2)            
y_train = keras.utils.to_categorical(y_train.T, 2)

#y_test = np.array(y_test, ndmin = 2)            
#y_test = keras.utils.to_categorical(y_test.T, 2)

In [9]:
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

In [13]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [8]:
from keras.callbacks import TensorBoard

In [9]:
import datetime

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [18]:
batch_size = 64
num_classes = 2
epochs = 5
#data_augmentation = True
#save_dir = os.path.join(os.getcwd(), 'saved_models')
#model_name = 'roadNN.h5'

In [15]:
from keras import regularizers

In [19]:
num_epochs = epochs

c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

print('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(y_train) if j[0] == 1]
idx1 = [i for i, j in enumerate(y_train) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print(len(new_indices))
print(x_train.shape)
x_train = x_train[new_indices, :, :, :]
y_train = y_train[new_indices]

train_size = y_train.shape[0]

c0 = 0
c1 = 0
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 63224 c1 = 63224
Balancing training data...
126448
(126448, 16, 16, 3)
Number of data points per class: c0 = 63224 c1 = 63224


In [20]:
model = Sequential() #create a model

model.add(Conv2D(16, (5, 5), activation = 'relu' , padding='same', #first conv + relu layer
                 data_format='channels_last',
                 input_shape=x_train.shape[1:]))
model.add(Conv2D(16, (5, 5), activation = 'relu' , padding='same', #first conv + relu layer
                 data_format='channels_last',
                 input_shape=x_train.shape[1:]))

model.add(MaxPooling2D(pool_size=(3, 3), strides=2, padding='same', data_format=None)) #first pooling layer

model.add(Conv2D(32, (5, 5), activation = 'relu' , padding='same')) #conv + relu layer
model.add(Conv2D(32, (5, 5), activation = 'relu' , padding='same')) #conv + relu layer

model.add(MaxPooling2D(pool_size=(3, 3), strides=2, padding='same', data_format=None)) #pooling layer


model.add(Flatten()) #flatt layer before feeding the fully connected layer
model.add(Dense(16, activity_regularizer=regularizers.l2()))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes)) #Fully Connected layer with 2 outputs
model.add(Activation('softmax') ) #sigmoid like, to set value between 0 and 1



In [21]:
model.compile(loss='binary_crossentropy', #compile the model
              optimizer='adam',
              metrics=['categorical_accuracy'])


model.fit(x_train, y_train,              #fitting the model 
              batch_size=batch_size,
              epochs=epochs,
              #validation_data=(x_test, y_test),
              #callbacks = [tensorboard_callback]
         )

Epoch 1/5
  9920/126448 [=>............................] - ETA: 2:58 - loss: 0.6895 - categorical_accuracy: 0.5556

KeyboardInterrupt: 

In [47]:
#print the loss and the metrics if using a split_data

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

from sklearn.metrics import f1_score

f1 = f1_score(y_t, pred, average='binary')  
print('F1 score:', f1)


82500/82500 [==============================] - 36s 438us/step
Test loss: 0.4207056113994483
Test accuracy: 0.8064969778060913
F1 score: 0.6741907832972774


In [23]:
y_t = np.zeros(y_test.shape[0])
y_t[y_test[:,1] == 1] = 1

In [16]:
sum(y_t)

5353.0

In [51]:
predictions = model.predict(y_test, batch_size=None, verbose=1, 
                            steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)
predictions.shape

72200/72200 [==============================] - 25s 349us/step


(72200, 2)

In [52]:
pred = np.zeros(predictions.shape[0]) 
pred[predictions[:,0]< predictions[:,1]] = 1 #merging predictions to 1d vector with value 0 or 1


In [54]:
create_submission(pred, "cnn_submission.csv")


In [17]:
model.save('cnn.model') #to load and save the current model 
#model = tf.keras.models.load_model('cnn.model')


In [ ]:
def postprocess(img):
    """ Postprocessing of the predictions
    Modify isolated road patchs to background """
    [dim_x, dim_y] = img.shape
    kernel = np.ones((3,3),np.float32)
    kernel[1,1] = 0

    filtered_img = signal.convolve2d(img, kernel,mode ="same",boundary = 'symm')
    postprocess_img = img.copy()

    for i in range(0, dim_y):
        for j in range(0, dim_x):
            if img[i,j] == 1:
                if filtered_img[i,j] < 2 :
                    # If a patch is predicted as road,
                    # but less than 2 neighbors are also predicted road :
                    # Then we consider the patch not to be road
                    postprocess_img[i,j] = 0

            #elif img[i,j] == 0:
                    # If a patch is predicted as NOT road,
                    # but more than 7 neighbors are predicted road :
                    # Then we consider the patch to be road
                #if filtered_img[i,j] >= 7 :
                   # postprocess_img[i,j] = 1

    return postprocess_img

In [ ]:
Z_eval = pred

# 
# Post processing on eval dataset
# 

# Reshape prediction as matrix for each image
z_reshaped = []
z_reshaped_pp = []

num_patch_total = len(Z_eval)
num_test = y_train.shape[0]

num_patch_by_img = num_patch_total // num_test

for i in range(0, num_patch_total, num_patch_by_img):
    z_crt = Z_eval[i : i + num_patch_by_img]
    z_crt_reshaped = np.reshape(z_crt, [608 // 16, 608 // 16])
    z_reshaped.append(z_crt_reshaped)
    z_reshaped_pp.append(np.reshape(postprocess(z_crt_reshaped), z_crt.shape[0]))

# # Run post process
# zzz = z_reshaped
# for ind, label_img in enumerate(z_reshaped):
#     label_img_pp = postprocess(label_img)
#     zzz[ind] = np.reshape(label_img_pp, [z_crt.shape[0]])

# Convert list as array
Z_eval_pp = np.concatenate( z_reshaped_pp , axis = 0 )
print("Postprocessing done!")

# 
# Save prediction
# 
create_submission(Z_eval_pp, "submission_LogReg_pp.csv")

In [ ]:
# Run prediction on the img_idx-th image
img_idx = 28

Xi = extract_img_features(image_dir + files[img_idx])
Xi = poly_augmentation(Xi, 2)
Zi = logreg.predict(Xi)

plt.scatter(Xi[:, 0], Xi[:, 1], c=Zi, edgecolors='k', cmap=plt.cm.Paired)

In [ ]:

img_idx = 28

w = gt_imgs[img_idx].shape[0]
h = gt_imgs[img_idx].shape[1]
predicted_im = label_to_img(w, h, patch_size, patch_size, Zi)
cimg = concatenate_images(imgs[img_idx], predicted_im)
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 
plt.imshow(cimg, cmap='Greys_r')

new_img = make_img_overlay(imgs[img_idx], predicted_im)

plt.imshow(new_img)